In [1]:
import pandas as pd
import numpy as np

import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Border, Side, Font

import warnings
warnings.filterwarnings('ignore')

In [2]:
from sqlite3 import connect
cnn = connect(r"drive/MyDrive/GENERATORY.db")
tab = pd.read_sql("SELECT * FROM baza_STOLARNIA", cnn)
cnn.close()

In [3]:
komplety_stelazy = pd.read_excel("drive/MyDrive/LISTA_KOMPLETOW_STELAZY.xlsx")

def pars_bryla(x):
  if type(x) != str:
    y = str(x)
    y = y.replace(".0", "").replace(".", ",")

    return y
  return x

komplety_stelazy["BRYLA_GEN"] = komplety_stelazy.BRYLA_GEN.apply(pars_bryla).astype(str)
# komplety_stelazy

##PRZYGOTOWANIE ZAMÓWIENIA

In [5]:
!rm -f *.xlsx

In [7]:
kompletacje = pd.read_excel("drive/MyDrive/ZAM_HORIZON_2_24_8.xlsx", sheet_name="Arkusz2")
kompletacje.rename(columns={"Unnamed: 0": "OPIS", kompletacje.columns[1]: "ILOSC", kompletacje.columns[2]: "KOMPLETACJA", kompletacje.columns[3]: "PALETA"}, inplace=True)

ostatnia_zajeta_paleta = 7503

In [8]:
kompletacje

,OPIS,ILOSC,KOMPLETACJA,PALETA
0,HORIZON 1z KO,15,1,1
1,"HORIZON 1,5z KO",15,1,1
2,"HORIZON 2,5z KO",35,1,2
3,"HORIZON 2,5z KO",15,1,3
4,HORIZON 3z KO,15,1,3
5,HORIZON 3z KO,35,1,4
6,"HORIZON 3,5z KO",35,1,5
7,"HORIZON 3,5z KO",35,1,6
8,HORIZON [LHz KOL,15,1,7
9,HORIZON L2Hz] KOP,10,1,7


In [9]:
kompl = kompletacje[kompletacje.KOMPLETACJA == 2]
kompl["PALETA"] = kompl.PALETA + ostatnia_zajeta_paleta

class Paleta:
  def __init__(self, model, nr_palety) -> None:
    self.model = model
    self.nr_palety = nr_palety
    pal = kompl[(kompl[kompl.columns[3]] == nr_palety)].merge(komplety_stelazy[["OPIS", "BRYLA_GEN"]], how="left", on="OPIS")

    paleta = pal.merge(tab[(tab.MODEL == model) & (tab.BRYLA.isin(pal.BRYLA_GEN.unique()))], how="left", right_on="BRYLA", left_on="BRYLA_GEN")
    paleta["ILOSC_ELEMENTOW"] = paleta.ILOSC * paleta._ILOSC

    def czas_kompl(czas_maszyny, ozn, ilosc):
      if czas_maszyny == 0:
        return 0
      if "MAG" in ozn:
        return 0
      return ilosc * 0.113

    paleta["CZAS_KOMPLETACJI"] = paleta.apply(lambda x: czas_kompl(x._CZAS, x.OZNACZ, x.ILOSC_ELEMENTOW), axis=1)
    paleta[["OPIS","MATERIAL", "NUMER", "ILOSC", "_ILOSC", "ILOSC_ELEMENTOW", "OZNACZ", "WYMIAR", "CZAS_KOMPLETACJI"]].sort_values(by=["MATERIAL"])

    podsumowanie_paleta = paleta[["MATERIAL", "NUMER", "ILOSC_ELEMENTOW","CZAS_KOMPLETACJI"]].groupby(["MATERIAL", "NUMER"]).sum().reset_index().merge(paleta[["MATERIAL", "NUMER", "WYMIAR", "OZNACZ"]].drop_duplicates(), how="left", on=["MATERIAL", "NUMER"])


    self.elementy = paleta
    self.podsumowanie = podsumowanie_paleta
    self.czas_kompletacji = podsumowanie_paleta.CZAS_KOMPLETACJI.sum()

  def Drukuj_raport(self, nr_zlecenia):
    wb = openpyxl.Workbook()
    sheet = wb.active

    ftd1 = Font(size=20, bold=True)
    a1 = sheet["A1"]
    a1.value = f"ZLECENIE NR: {nr_zlecenia}"
    a1.font = ftd1
    sheet.merge_cells('A1:G1')

    sheet["A3"] = "SERIA"
    sheet["A4"] = "WÓZEK"
    sheet["A5"] = "NR SYMB"
    sheet["A6"] = "DATA WYDANA"
    sheet["A7"] = "DATA ZAKOŃCZENIA"
    sheet["C3"] = f"{self.model}"
    sheet["C4"] = f"Wózek nr {self.nr_palety}"
    sheet["C5"] = f"{self.model[:3]}"
    thin = Side(border_style="thin", color="000000")
    for row in sheet[f"A3:D7"]:
      for cell in row:
        cell.border = Border(bottom=thin)



    sheet["A48"] = f"{self.model}"
    sheet["A49"] = f"Zestawienie dla serii {self.model} w zleceniu nr : {nr_zlecenia}"
    a =  "  ".join([f"{x[1].ILOSC}x {x[1].OPIS.replace(self.model, '').strip()}" for x in kompl[kompl.PALETA == self.nr_palety][["OPIS", "ILOSC"]].iterrows()])
    sheet["A51"] = f"Wózek nr : {self.nr_palety} - {self.model[:3]}: " + a

    sheet["A53"] = "LP"
    sheet["B53"] = "MATERIAL"
    sheet["C53"] = "NUMER"
    sheet["D53"] = "ILOSC"
    sheet["E53"] = "WYMIAR"
    sheet["F53"] = "OZNACZ"
    sheet["G53"] = "CZAS_KOMPL"
    DF = self.podsumowanie[["MATERIAL", "NUMER", "ILOSC_ELEMENTOW", "WYMIAR", "OZNACZ", "CZAS_KOMPLETACJI"]]
    rows = dataframe_to_rows(DF, header=False)#.reindex(pd.Index([x for x in range(1,self.podsumowanie.shape[0]+1)])),

    for r_idx, row in enumerate(rows,1):
       for c_idx, value in enumerate(row,1):
             sheet.cell(row=r_idx+52, column=c_idx, value=value)

    lp = 1
    for i in range(DF.shape[0]):
      sheet.cell(row=54+i, column=1, value=lp)
      lp += 1

    thin = Side(border_style="thin", color="000000")
    for row in sheet[f"A53:g{DF.shape[0]+53}"]:
        for cell in row:
          cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)

    sheet[f"D{DF.shape[0]+54}"] = DF.ILOSC_ELEMENTOW.sum()
    sheet[f"G{DF.shape[0]+54}"] = DF.CZAS_KOMPLETACJI.sum()

    ft1 = Font(size=100, bold=True)
    d120 = sheet["B121"]
    d121 = sheet["B122"]
    d120.value = f"{self.model}"
    d121.value = f"{self.nr_palety}"
    d120.font = ft1
    d121.font = ft1

    wb.save(f"{nr_zlecenia.replace('/', '_')} {self.nr_palety}.xlsx")

  def __repr__(self):
    return f"paleta nr: {self.nr_palety} {self.czas_kompletacji:.0f}min"


In [10]:
kompl

,OPIS,ILOSC,KOMPLETACJA,PALETA
16,HORIZON 1z OP,15,2,7504
17,"HORIZON 1,5z OP",10,2,7504
18,"HORIZON 2,5z OP",10,2,7504
19,HORIZON 3z OP,45,2,7505
20,HORIZON 3z OP,15,2,7506
21,"HORIZON 3,5z OP",20,2,7506
22,"HORIZON 3,5z OP",45,2,7507
23,HORIZON [LHz OPL,15,2,7508
24,HORIZON L2Hz] OPP,10,2,7508
25,HORIZON LHz] OPP,30,2,7509


In [11]:
palety = [Paleta("HORIZON", p) for p in kompl.PALETA.drop_duplicates().to_list()]
palety

[paleta nr: 7504 63min,
 paleta nr: 7505 92min,
 paleta nr: 7506 71min,
 paleta nr: 7507 92min,
 paleta nr: 7508 72min,
 paleta nr: 7509 85min,
 paleta nr: 7510 64min,
 paleta nr: 7511 81min,
 paleta nr: 7512 68min,
 paleta nr: 7513 41min,
 paleta nr: 7514 56min,
 paleta nr: 7515 90min,
 paleta nr: 7516 36min]

In [12]:
sum([x.czas_kompletacji for x in palety])

910.7799999999999

In [13]:
palety[-1].elementy

,OPIS,ILOSC,KOMPLETACJA,PALETA,BRYLA_GEN,MODEL,MATERIAL,WYMIAR,_CZAS,_ILOSC,BRYLA,NUMER,OZNACZ,WYS,SZER,DLUG,ILOSC_ELEMENTOW,CZAS_KOMPLETACJI
0,HORIZON MAX [,20,2,7516,MAX ][,HORIZON,Sosna,25x25x948,18,2,MAX ][,0700,OP,25,25.0,948.0,40,4.52
1,HORIZON MAX [,20,2,7516,MAX ][,HORIZON,Sosna,25x25x350,18,1,MAX ][,2174,OP,25,25.0,350.0,20,2.26
2,HORIZON MAX [,20,2,7516,MAX ][,HORIZON,Sosna,25x50x785,18,1,MAX ][,2753,OP,25,50.0,785.0,20,2.26
3,HORIZON MAX [,20,2,7516,MAX ][,HORIZON,Sosna,25x50x948,18,1,MAX ][,2754,OP,25,50.0,948.0,20,2.26
4,HORIZON MAX [,20,2,7516,MAX ][,HORIZON,Płyta wiórowa 16mm,*** 182x520,26,1,MAX ][,1776,CN,15,182.0,520.0,20,2.26
5,HORIZON MAX [,20,2,7516,MAX ][,HORIZON,Płyta wiórowa 16mm,*** 200x520,26,1,MAX ][,1777,CN,15,200.0,520.0,20,2.26
6,HORIZON MAX [,20,2,7516,MAX ][,HORIZON,Płyta wiórowa 16mm,*** 200x520,26,1,MAX ][,1778,CN,15,200.0,520.0,20,2.26
7,HORIZON MAX [,20,2,7516,MAX ][,HORIZON,Płyta wiórowa 16mm,*** 200x980,26,1,MAX ][,1779,CN,15,200.0,980.0,20,2.26
8,HORIZON MAX [,20,2,7516,MAX ][,HORIZON,Płyta wiórowa 16mm,150x400,26,1,MAX ][,1780,PP,15,150.0,400.0,20,2.26
9,HORIZON MAX [,20,2,7516,MAX ][,HORIZON,Płyta wiórowa 16mm,200x520,26,1,MAX ][,1781,PP,15,200.0,520.0,20,2.26


In [14]:
for p in palety:
  p.Drukuj_raport("HOR 2/2/24")